In [11]:
import pandas as pd
import numpy as np


In [20]:
df = pd.read_csv('house_features.csv')
df.describe()

,Room,Price,Price(USD)
count,3479.000000,3.479000e+03,3.479000e+03
mean,2.079908,5.359023e+09,1.786341e+05
std,0.758275,8.099935e+09,2.699978e+05
min,0.000000,3.600000e+06,1.200000e+02
25%,2.000000,1.418250e+09,4.727500e+04
50%,2.000000,2.900000e+09,9.666667e+04
75%,2.000000,6.000000e+09,2.000000e+05
max,5.000000,9.240000e+10,3.080000e+06


In [26]:
data = pd.read_csv('house_features.csv')

data = data[data.apply(lambda row: len(row) == 8, axis=1)]
data = data[len(data['Area']) <= 1000]

data = data[data['Address'].notnull()]

data.to_csv('cleaned_dataset.csv', index=False)
cleaned_data_set = pd.read_csv('cleaned_dataset.csv')
cleaned_data_set

KeyError: False